# Make a coastal sfincs model from scratch using hydromt

This notebook demonstrates how to prepare and inspect a SFINCS model from scratch. The notebook does not contain the SFINCS code or executables to run the model with. 

We will make a schematization with the following features:
* we only simulate coastal flooding, we do not consider any interactions with riverine flooding
* we use the MERIT Hydro dataset as elevation
* we include boundary conditions from the Global Tide and Surge Model by imposing a number of boundary points    where forcing is expected



All lines in this notebook which start with `!` are executed from the command line. Within the notebook environment the logging messages are shown after completion. You can also copy these lines and paste these in your shell to get more direct feedback.

### HydroMT CLI build interface


First let us see what hydromt provides us with.

In [ ]:
!hydromt --help

Whith the `models` flag we can ask if hydromt recognizes the SFINCS plugin (sfincs should be mentioned):

In [ ]:
!hydromt --models

It looks like we have several commands that we can consider. We want to `build` a model. How does that work? We can also call `--help` on the available hydromt commands as follows.

In [ ]:
!hydromt build --help

### Building a first model (and getting an error when data is missing)
Let's try to build an entire schematisation, just from the command line (Awesome!!!). In the `--help` section, there is an example for making a model from a bounding box, which is very intuitive. Let's try exactly that. We store the model on the local path `./texel_sfincs`

In [ ]:
!hydromt build sfincs ./texel_sfincs "{'bbox': [4.6891,52.9750,4.9576,53.1994]}"

The above might have taken a few seconds, but then should have miserably crashed! What happened:
* HydroMT was looking for a data catalog to use. 
* It did not find any and therefore, it automatically downloaded a sample data catalog which you can now find on your home folder under .hydromt_data
* It tried to build a model in the Texel bounding box, but...alas, the sample dataset does not cover Texel at all! To add datasets covering the area outside the sample data extent, setup your own datasets using data yml files [hydromt data](https://deltares.github.io/hydromt/latest/user_guide/data.html) using `--data PATH` option, or if your are connected to the deltares network `--deltares-data` flag.


### Building a first model (and getting it right)
Let's adapt the bounding box to an area that fits within the sample dataset. The dataset covers the Piave basin and its surroundings in Italy. A good place to get a CSV formatted bounding box is https://boundingbox.klokantech.com/

We also add `-vv` for extra verbosity

In [ ]:
!hydromt build sfincs ./venice_sfincs "{'bbox': [12.047909,45.15809,12.833432,45.646944]}" -vv

This should work without any errors. 

The example above means the following: run **hydromt build** with:

* `sfincs` : i.e. build a sfincs model
* `./venice_sfincs` : output model folder
* `"{'bbox': [12.047909,45.15809,12.833432,45.646944]}"` : make a model that is active for the full given bounding box. All *REGION* options are described in the [docs](https://deltares.github.io/hydromt/latest/user_guide/cli.html#region-options)
* `-vv` : give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.

NOTE: As we did not specify a model configuration, only the base maps (topography) have been setup using default parameters. To build a complete model we need the use a configuraton ini-file.

We can now inspect the newly created folder `./venice_sfincs`

In [ ]:
!ls ./venice_sfincs

The .log file contains the same messages as shown in the verbose messages during the building of the model. The files are standard sfincs formats and contain the following
* .inp file : holds the model's configuration
* .dep file : holds the bathymetry in the typical format of sfincs
* .msk file : mask file (0, inactive cells, 1, active cells, 2, boundary cells)

Let us have a closer look at the model's configuration

In [ ]:
fn_inp = './venice_sfincs/sfincs.inp'
with open(fn_inp, 'r') as f:
    txt = f.read()
print(txt)

We see that quite standard settings are used. The coordinate reference system (see `epsg`) is automatically chosen to the nearest UTM zone. We have not even considered the resolution. It is now 100 meters (see `dx` and `dy`). Let's assume we want a 150 meter model and rebuild it. The `--help` section showed that the `-r` option has to be used to change this.

In [ ]:
!hydromt build sfincs ./venice_sfincs_150 "{'bbox': [12.047909,45.15809,12.833432,45.646944]}" -r 150 -vv

In [ ]:
fn_inp = './venice_sfincs_150/sfincs.inp'
with open(fn_inp, 'r') as f:
    txt = f.read()

print(txt)

It is clear that the `dx` and `dy` settings are changed.

### Building a complete model (ready to run)

We can provide many more settings in an confuration (.ini) file instead of command line arguments. If you don't provide an ini file (as done above), only the basemaps component is build with default settings. To complete your model you can add more [model components](https://deltares.github.io/hydromt_sfincs/latest/user_guide/sfincs.html) to the model setup.

Let's have a look a typical ini file for setting up coastal sfincs model, where each `[setup_<>]` refers to a model component. All options which end with `_fn` refers to a data source from the data catalog or a path to data on your machine, see [hydromt data](https://deltares.github.io/hydromt/latest/user_guide/data.html). In this case some sample data which is downloaded to your machine at `$USER_ROOT/.hydromt_data`

In [ ]:
fn_ini = 'sfincs_coastal.ini'
with open(fn_ini, 'r') as f:
    txt = f.read()
print(txt)

You can see the following sections:
* `[global]`: arguments passed to SfincsModel initialization. With the `write_gis` flag a copy of the model schmatization in GIS compatible formats is saved in the gis subfolder. 
* `[setup_config]`: arguments which are forwarded to the scinf model configuration file sfincs.inp
* `[setup_basemaps]`: basic information required for the elevation, bathymetry, land mask. These are the typical demanded static layers that any sfincs model needs. Here you may for instance consider altering the elevation and bathymetry data for local data sources. These then have to be supplied in the `data_catalog.yml`.
* `[setup_cn_infiltration]`: infiltration curve number maps, see [sfincs docs](https://sfincs.readthedocs.io/en/latest/input.html?highlight=curve#spatially-varying-curve-number) for more information
* `[setup_manning_roughness]`: configuration of the Manning roughness values, typically consisting of a land use or land cover map, and a lookup table that relates land use classes to manning roughness values in sec m$^{-1/3}$
* `[setup_h_forcing]`: water level forcing data. This can be provided as a NetCDF file following CF-conventions for point time series data.
* `[setup_p_forcing_gridded]`: gridded precipitation data, that can be used to do compound flood simulations. The `[setup_cn_infiltration]` is then quite essential to include.

To build a model from scratch add the ini file to the command line `hydromt build` method 

In [ ]:
!hydromt build sfincs ./venice_sfincs_150_full "{'bbox': [12.047909,45.15809,12.833432,45.646944]}" -r 150 -i sfincs_coastal.ini -vv

In [ ]:
!ls ./venice_sfincs_150_full 

### Inspecting the model
Let's have a closer look at the model. To interact with a model within a notebook or python script, we need to load a number of modules.

In [ ]:
%matplotlib inline
import hydromt
import os

Now we can load the model in memory and inspect it. 

Note that based on the global datasets the lagoon bathymetry is not well represented. A better merge between MERIT hydro topography and GEBCO bathymetry can be achieved by providing an better coastline shapefile (by default OSM based) which determines which dataset is used where.

In [ ]:
# define the model's configuration file
root = r'./venice_sfincs_150_full'

# read the model with hydromt sfincs methods
mod = hydromt.SfincsModel(root=root, mode="r")
mod.read()
mod.plot_basemap(figsize=(12, 8))

As said above, we can seriously and very easily improve the above result, by introducing a better land boundary. You can draw one yourself in GIS and insert a geojson, geopackage or shapefile with a relative or absolute path under `landmask_fn`. In the following .ini file, we are referring to a geopackage file. We also set `dep_contour = -5` to ensure that deeper cells also get activated in the sfincs model. Let's rebuild the model with this .ini file.

In [ ]:
fn_ini = 'sfincs_coastal_improved_lb.ini'
with open(fn_ini, 'r') as f:
    txt = f.read()
print(txt)

In [ ]:
!hydromt build sfincs ./venice_sfincs_150_full_improved_lb "{'bbox': [12.047909,45.15809,12.833432,45.646944]}" -r 150 -i sfincs_coastal_improved_lb.ini -vv

Let's finally inspect the result again

In [ ]:
# define the model's configuration file
import hydromt
root = r'./venice_sfincs_150_full_improved_lb'

# read the model with hydromt sfincs methods
mod = hydromt.SfincsModel(root=root, mode="r")
mod.read()
mod.plot_basemap(figsize=(12, 8))

### Next steps
You can update an existing model, e.g. to replace parameterisations or add forcing. this is descibed in a follow-up notebook `sfincs_update_model.ipynb`